<a href="https://colab.research.google.com/github/lmoroney/tfbook/blob/master/tfrecords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Good to run this to ensure you are using TF2.x

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [5]:
print(tfds.__version__)

2.0.0


In [6]:
# At time of creation, tfds was at version 2.0 which had
# some bugs in some common datasets. Advise to update to
# tfds 2.1.0 like this:
pip install tensorflow_datasets==2.1.0

     |████████████████████████████████| 3.1MB 4.7MB/s 
ERROR: tensor2tensor 1.14.1 has requirement tensorflow-probability==0.7.0, but you'll have tensorflow-probability 0.9.0 which is incompatible.
  Found existing installation: tensorflow-datasets 2.0.0
    Uninstalling tensorflow-datasets-2.0.0:
      Successfully uninstalled tensorflow-datasets-2.0.0


In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
print(tfds.__version__)
data, info = tfds.load("cnn_dailymail:1.0.0", with_info=True)

2.1.0


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/1.0.0.incompleteFFCKFG/cnn_dailymail-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/1.0.0.incompleteFFCKFG/cnn_dailymail-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/1.0.0.incompleteFFCKFG/cnn_dailymail-test.tfrecord



Dataset cnn_dailymail downloaded and prepared to /root/tensorflow_datasets/cnn_dailymail/plain_text/1.0.0. Subsequent calls will reuse this data.


#First Example: Simple dataset
Let's look at how to use a simple dataset like Fashion MNIST

In [0]:
# Install: pip install tensorflow-datasets
import tensorflow as tf
import tensorflow_datasets as tfds
(training_images, training_labels), (test_images, test_labels) =  tfds.as_numpy(tfds.load('fashion_mnist',split = ['train', 'test'], batch_size=-1, as_supervised=True))

training_images = training_images / 255.0
test_images = test_images / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28,28,1)),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


model.fit(training_images, training_labels, epochs=5)

Code to install tensorflow addons if we are going to use any of its
libraries for augmentation

In [0]:
pip install tensorflow-addons

# Second Example: Horses or Humans with Validation
This will use Horses or Humans with validation to train with the
data loaded from TFDS

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

data = tfds.load('horses_or_humans', split='train', as_supervised=True)
val_data = tfds.load('horses_or_humans', split='test', as_supervised=True)

def augmentimages(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/255)
  image = tf.image.random_flip_left_right(image)
  image = tfa.image.rotate(image, 40, interpolation='NEAREST')
  return image, label

train = data.map(augmentimages)
train_batches = train.shuffle(100).batch(32)
validation_batches = val_data.batch(32)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', 
                                      input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_batches, epochs=10, validation_data=validation_batches, validation_steps=1)

# Third Example: Cats v Dogs with custom splits


In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import numpy as np


def augmentimages(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/255)
  image = tf.image.resize(image,(300,300))
  return image, label

count_data = tfds.load('cats_vs_dogs',split='train',as_supervised=True)
train_data = tfds.load('cats_vs_dogs', split='train[:80%]', as_supervised=True)
validation_data = tfds.load('cats_vs_dogs', split='train[80%:90%]', as_supervised=True)
test_data = tfds.load('cats_vs_dogs', split='train[-10%:]', as_supervised=True)


#count_length = [i for i,_ in enumerate(count_data)][-1] + 1
#print(count_length)

#train_length = [i for i,_ in enumerate(train_data)][-1] + 1
#print(train_length)

#validation_length = [i for i,_ in enumerate(validation_data)][-1] + 1
#print(validation_length)

#test_length = [i for i,_ in enumerate(test_data)][-1] + 1
#print(test_length)

augmented_training_data=train_data.map(augmentimages)
augmented_validation_data=validation_data.map(augmentimages)
train_batches = augmented_training_data.shuffle(100).batch(10)
validation_batches = augmented_validation_data.batch(10)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', 
                                      input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_batches, epochs=25, validation_data=validation_batches, validation_steps=1)